<a href="https://colab.research.google.com/github/rachel-ker/refugeediscourse/blob/master/BERT_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Final Project - Deep Neural Networks


In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [3]:
!pip install transformers

In [4]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
data = pd.read_csv("./refugee_coca_foranalysis.csv")

In [0]:
#5-year periods
def get_period(year, startyr, endyr, n=5):
    period_start = []
    for i in range(startyr, endyr+1, n):
        period_start.append(i)
    for index, p in enumerate(period_start):
        if year >= p:
            period = index
            continue
        else:
            break
    return period  
    
data['period'] = data['year'].apply(lambda x: get_period(x, 1991, 2015, n=5) if x>=1991 else 0)

In [0]:
import re

# Add label
unique_sources = list(data.source.apply(lambda x: re.split("_|: | ", x)[0]).unique())
len(unique_sources)

# Write a function that let us conveniently label ideology: Left, Neutral, Right
media_ideology = {}

def label_ideology(media_title, ideology, media_ideology = media_ideology):
    media_ideology[media_title] = ideology
    
label_ideology('Money', 'Neutral')
label_ideology('MotherEarth', 'Neutral')
label_ideology('MotherJones', 'Left')
label_ideology('AmHeritage', 'Neutral')
label_ideology('AmSpect', 'Right')
label_ideology('Forbes', 'Right')
label_ideology('NatlReview', 'Right')
label_ideology('Newsweek', 'Left')
label_ideology('ScienceNews', 'Neutral')
label_ideology('Smithsonian', 'Neutral')
label_ideology('USNWR', 'Left')
label_ideology('WashMonth', 'Left')
label_ideology('ChangingTimes', 'Right')
label_ideology('HistoryToday', 'Neutral')
label_ideology('Omni', 'Neutral')
label_ideology('Wilderness', 'Neutral')
label_ideology('TIME', 'Left')
label_ideology('NatlParks', 'Neutral')
label_ideology('AmerArtist', 'Neutral')
label_ideology('RollingStone', 'Left')
label_ideology('Americas', 'Neutral')
label_ideology('SportsIll', 'Neutral')
label_ideology('Ms', 'Left')
label_ideology('PopScience', 'Neutral')
label_ideology('Futurist', 'Neutral')
label_ideology('HarpersMag', 'Left')
label_ideology('Fortune', 'Right')
label_ideology('USAToday', 'Left')
label_ideology('America', 'Left')
label_ideology('ChristCentury', 'Right')
label_ideology('People', 'Left')
label_ideology('Jet', 'Left')
label_ideology('Aging', 'Neutral')
label_ideology('Horticulture', 'Neutral')
label_ideology('NewRepublic', 'Left')
label_ideology('Conservation', 'Left')
label_ideology('NaturalHist', 'Neutral')
label_ideology('Atlantic', 'Left')
label_ideology('Inc.', 'Neutral')
label_ideology('ChildrenToday', 'Neutral')
label_ideology('Ebony', 'Left')
label_ideology('ConsumResrch', 'Neutral')
label_ideology('SatEvenPost', 'Neutral')
label_ideology('ChristToday', 'Right')
label_ideology('Backpacker', 'Neutral')
label_ideology('AmericanCraft', 'Neutral')
label_ideology('ArtAmerica', 'Neutral')
label_ideology('SportingNews', 'Neutral')
label_ideology('MensHealth', 'Neutral')
label_ideology('Antiques', 'Neutral')
label_ideology('Parenting', 'Neutral')
label_ideology('Essence', 'Neutral')
label_ideology('Environmental', 'Neutral')
label_ideology('USCatholic', 'Right')
label_ideology('MilitaryHist', 'Neutral')
label_ideology('PsychToday', 'Neutral')
label_ideology('Cosmopolitan', 'Left')
label_ideology('Redbook', 'Neutral')
label_ideology('Bazaar', 'Left')
label_ideology('ChildDigest', 'Neutral')
label_ideology('Bicycling', 'Neutral')
label_ideology('Shape', 'Neutral')
label_ideology('NatGeog', 'Neutral')
label_ideology('Entertainment', 'Neutral')
label_ideology('Astronomy', 'Neutral')
label_ideology('TownCountry', 'Neutral')
label_ideology('TotalHealth', 'Neutral')
label_ideology('Esquire', 'Left')
label_ideology('FieldStream', 'Neutral')
label_ideology('TechReview', 'Neutral')
label_ideology('CountryLiving', 'Neutral')
label_ideology('VegTimes', 'Neutral')
label_ideology('SouthernLiv', 'Neutral')
label_ideology('Skiing', 'Neutral')
label_ideology('ConsumRep', 'Neutral')
label_ideology('Sunset', 'Neutral')
label_ideology('HarpersBazaar', 'Neutral')
label_ideology('AmericanSpectator', 'Right')
label_ideology('GoodHousekeeping', 'Neutral')
label_ideology('PopMech', 'Neutral')
label_ideology('MHQTheQuarterly', 'Neutral')
label_ideology('TodaysParent', 'Neutral')
label_ideology('NationalGeographic', 'Neutral')
label_ideology('EEnvironmental', 'Neutral')
label_ideology('ParentingEarly', 'Neutral')
label_ideology('ABC', 'Left')
label_ideology('CNN', 'Left')
label_ideology('PBS', 'Left')
label_ideology('CBS', 'Left')
label_ideology('Ind', 'Left')
label_ideology('NPR', 'Left')
label_ideology('NBC', 'Left')
label_ideology('Fox', 'Right')
label_ideology('MSNBC', 'Left')
label_ideology('NYTimes', 'Left')
label_ideology('CSMonitor', 'Neutral')
label_ideology('AssocPress', 'Neutral')
label_ideology('WashPost', 'Left')
label_ideology('SanFranChron', 'Left')
label_ideology('Atlanta', 'Left') #Atlanta Journal Constitution
label_ideology('Houston', 'Left') #Houston Chronicle
label_ideology('Chicago', 'Left') #Chicago Sun-Times
label_ideology('Denver', 'Left') #Denver Post
label_ideology('GolfMag', 'Neutral')
label_ideology('NewStatesman', 'Left')
label_ideology('Austin', 'Left') #Austin American Statesman
label_ideology('STLouis', 'Left') #St Louis Post_Dispatch
label_ideology('Pittsburgh', 'Right') #Pittsburgh Post-Gazette
label_ideology('OrangeCR', 'Right') #Orange County Register

#add political leaning label
def add_ideology(x, media_ideology=media_ideology):
    try_split = re.split("_|: | ",x)
    if len(try_split)>1:
        x = try_split[0]
    if x in media_ideology:
        return media_ideology[x]
    else:
        print('{} does not exists'.format(x))

data["ideology"] = data['source'].apply(add_ideology)

In [0]:
data = data[data['ideology']!='Neutral']
data['label'] = data['ideology'].mask(data['ideology']=='Right',1)
data['label'] = data['label'].mask(data['ideology']=='Left',0)

In [9]:
data.groupby(['ideology','label']).size()

ideology  label
Left      0        2728
Right     1         271
dtype: int64

In [10]:
data = data[['text','label','period']]
data.rename(columns={'text':'sentence'}, inplace=True)
data.head()

,sentence,label,period
3,section : movements from socialist to republi...,0,0
4,inside a dusty cement-block house with worn ...,0,0
6,even if saddam hussein fails to get his way ...,1,0
7,section : spectator 's journal dateline : li...,1,0
8,soviet jewry is giving israel new life -- bu...,1,0


In [0]:
df_test = data[data.period==4]
df_train = data[data.period!=4]

In [63]:
df_train.sample(10)

,sentence,label,period
3001,a snicket ticket could have been a people pl...,0,2
1587,host : oprah winfrey !executive-producer : di...,0,2
1866,!robert-siegel-hos : this is all things consi...,0,3
1532,"11:00 am-12:00 noon , many afghans living in ...",0,2
3219,what is the origin of road movies ? a year a...,0,3
453,the company has been walloped by terror and r...,1,2
1490,"colleen mcedwards , world news ( voice-over )...",0,1
2732,"istanbul , turkey - terrified earthquake sur...",0,1
1260,"peter jennings , abc news : voice-over in los...",0,1
670,"in july , lebanon 's leadership finally forme...",0,3


In [64]:
df_test.sample(10)

,sentence,label,period
3914,!george-stephanopou# (off-camera) let 's brin...,0,4
3786,ethical failure # basic ethics should n't hav...,1,4
739,"he 's not here in "" welcome to the cafeteria ...",0,4
3575,: views what are we to make of the horrific e...,1,4
3984,!gwen-ifill# president obama said today gun c...,0,4
3697,"!chuck-todd , -nbc-ne# host : this sunday on ...",0,4
3979,!jeffrey-brown# police and protesters clashed...,0,4
3434,"seoul , south korea # in august 1996 , the ve...",0,4
3401,"on tuesday in a ceremony in rome , the united...",0,4
3882,"!donald-trump-(r) , -# we will have so much w...",1,4


### LSTM (First 4 periods predicting the last period)

In [0]:
# Create sentence and label lists
sentences = df_train.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df_train.label.values

In [66]:
sentences[:5]

['[CLS]  section : movements from socialist to republicans , a generation of student activists test the limits of volunteerism .  university of houston student lloyd jacobson was organizing for a sleep-in outside a local homeless shelter and invited the head of the campus college republicans to join in . " do n\'t you want to be part of bush \'s thousand points of light ? " jacobson asked , and quickly got the student to agree . others in the republican group hesitated , but reluctantly came along . jacobson next enlisted the college democrats : " they \'re participating . are n\'t you going to as well ? "  the houston students initially regarded homeless people as faceless and disposable . but they were unnerved by what they witnessed while trying to sleep on cardboard sheets on the hard , cold ground . huddled together , they gingerly began talking with shelter residents . the young republican coordinator returned to write an editorial for the campus daily paper , in which he said he

In [67]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'section', ':', 'movements', 'from', 'socialist', 'to', 'republicans', ',', 'a', 'generation', 'of', 'student', 'activists', 'test', 'the', 'limits', 'of', 'volunteer', '##ism', '.', 'university', 'of', 'houston', 'student', 'lloyd', 'jacobs', '##on', 'was', 'organizing', 'for', 'a', 'sleep', '-', 'in', 'outside', 'a', 'local', 'homeless', 'shelter', 'and', 'invited', 'the', 'head', 'of', 'the', 'campus', 'college', 'republicans', 'to', 'join', 'in', '.', '"', 'do', 'n', "'", 't', 'you', 'want', 'to', 'be', 'part', 'of', 'bush', "'", 's', 'thousand', 'points', 'of', 'light', '?', '"', 'jacobs', '##on', 'asked', ',', 'and', 'quickly', 'got', 'the', 'student', 'to', 'agree', '.', 'others', 'in', 'the', 'republican', 'group', 'hesitated', ',', 'but', 'reluctantly', 'came', 'along', '.', 'jacobs', '##on', 'next', 'enlisted', 'the', 'college', 'democrats', ':', '"', 'they', "'", 're', 'participating', '.', 'are', 'n', "'", 't', 'you', 'going', 'to', 'a

In [0]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

In [0]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [0]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [73]:
print(len(df_train), len(input_ids), len(attention_masks))

2485 2485 2485


In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2020, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2020, test_size=0.1)

In [0]:
vocab_in_size = tokenizer.vocab_size
embedding_dim = 32
unit = 100
no_labels = len(np.unique(train_labels))
batch_size = 32

In [77]:
# Using Softmax activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='softmax'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 128, 32)           976704    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 1,030,106
Trainable params: 1,030,106
Non-trainable params: 0
_________________________________________________________________


In [84]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Train on 2236 samples, validate on 249 samples
Epoch 1/10
2236/2236 [==============================] - 14s 6ms/step - loss: 1.1809e-04 - acc: 1.0000 - val_loss: 0.7248 - val_acc: 0.9076
Epoch 2/10
2236/2236 [==============================] - 14s 6ms/step - loss: 7.4625e-05 - acc: 1.0000 - val_loss: 0.7633 - val_acc: 0.9036
Epoch 3/10
2236/2236 [==============================] - 14s 6ms/step - loss: 5.9153e-05 - acc: 1.0000 - val_loss: 0.7979 - val_acc: 0.9036
Epoch 4/10
2236/2236 [==============================] - 14s 6ms/step - loss: 4.9050e-05 - acc: 1.0000 - val_loss: 0.8266 - val_acc: 0.9036
Epoch 5/10
2236/2236 [==============================] - 15s 6ms/step - loss: 4.2048e-05 - acc: 1.0000 - val_loss: 0.8510 - val_acc: 0.9036
Epoch 6/10
2236/2236 [==============================] - 14s 6ms/step - loss: 3.6717e-05 - acc: 1.0000 - val_loss: 0.8711 - val_acc: 0.9036
Epoch 7/10
2236/2236 [==============================] - 13s 6ms/step - loss: 3.2580e-05 - acc: 1.0000 - val_loss: 0.889

In [87]:
# Create sentence and label lists
sentences = df_test.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df_test.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'calls', 'for', 'the', 'creation', 'of', 'a', 'no', '-', 'fly', 'zone', 'over', 'libya', 'grew', 'louder', 'in', 'the', 'united', 'states', 'as', 'civilians', 'came', 'under', 'fire', 'from', 'forces', 'loyal', 'to', 'the', 'libyan', 'leader', '.', 'col', '.', 'mu', '##am', '##mar', 'e', '##i', '-', 'q', '##ad', '##da', '##fi', ',', 'and', 'opposition', 'forces', 'suffered', 'a', 'number', 'of', 'battlefield', 'set', '##backs', '.', 'over', 'the', 'course', 'of', 'a', 'handful', 'of', 'brutal', 'days', 'in', 'early', 'march', ',', 'pro', '-', 'q', '##ad', '##da', '##fi', 'forces', 'reclaimed', 'communities', 'and', 'much', 'of', 'an', 'oil', 'facility', 'that', 'had', 'been', 'under', 'control', 'of', 'q', '##ad', '##da', '##fi', "'", 's', 'lightly', 'armed', 'resistance', '.', 'the', 'poorly', 'organized', 'opposition', 'force', ',', 'which', 'first', 'emerged', 'as', 'a', 'peaceful', 'protest', 'against', 'q', '##ad', '##da', '##fi', "'", 's', '

In [106]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)

514/514 [==============================] - 1s 2ms/step


In [107]:
evaluate_lstm
#returns loss value & metrics values

[1.8203000056938976, 0.8035019455252919]

In [110]:
# Using Sigmoid Activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='sigmoid'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 128, 128)          3906816   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               91600     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 3,998,618
Trainable params: 3,998,618
Non-trainable params: 0
_________________________________________________________________


In [111]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Train on 2236 samples, validate on 249 samples
Epoch 1/10
2236/2236 [==============================] - 14s 6ms/step - loss: 0.3242 - acc: 0.9217 - val_loss: 0.2705 - val_acc: 0.9237
Epoch 2/10
2236/2236 [==============================] - 13s 6ms/step - loss: 0.2543 - acc: 0.9293 - val_loss: 0.2803 - val_acc: 0.9237
Epoch 3/10
2236/2236 [==============================] - 14s 6ms/step - loss: 0.2354 - acc: 0.9293 - val_loss: 0.3083 - val_acc: 0.9237
Epoch 4/10
2236/2236 [==============================] - 14s 6ms/step - loss: 0.1140 - acc: 0.9517 - val_loss: 0.4009 - val_acc: 0.9076
Epoch 5/10
2236/2236 [==============================] - 14s 6ms/step - loss: 0.0201 - acc: 0.9928 - val_loss: 0.5101 - val_acc: 0.8996
Epoch 6/10
2236/2236 [==============================] - 14s 6ms/step - loss: 0.0074 - acc: 0.9987 - val_loss: 0.5673 - val_acc: 0.8956
Epoch 7/10
2236/2236 [==============================] - 14s 6ms/step - loss: 0.0040 - acc: 0.9982 - val_loss: 0.6022 - val_acc: 0.9157
Epoch 8/

In [112]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)
evaluate_lstm

514/514 [==============================] - 1s 3ms/step


[1.5126004551924856, 0.8015564202334631]

## Train and test for each year

#### Period 0

In [115]:
period0 = data[data.period==0]
period0

,sentence,label,period
3,section : movements from socialist to republi...,0,0
4,inside a dusty cement-block house with worn ...,0,0
6,even if saddam hussein fails to get his way ...,1,0
7,section : spectator 's journal dateline : li...,1,0
8,soviet jewry is giving israel new life -- bu...,1,0
...,...,...,...
2477,they signed up with human smugglers in their...,0,0
2478,moscow -- russia 's disastrous war in chechn...,0,0
2479,robert s. mcnamara 's recently expressed reg...,0,0
2480,the stories that fill our lives rarely make ...,0,0


In [161]:
period0.label.sum()/len(period0)
# number of right wing media

0.06019007391763464

In [0]:
period0_train, period0_test = train_test_split(period0, random_state=2020, test_size=0.05)

In [122]:
print(len(period0_train), len(period0_test))

899 48


In [123]:
# Create sentence and label lists
sentences = period0_train.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period0_train.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'prime', 'minister', 'yi', '##tz', '##hak', 'sham', '##ir', 'stressed', 'today', 'that', 'israel', 'should', 'seize', 'the', 'opportunity', 'to', 'attend', 'the', 'american', '-', 'designed', 'peace', 'conference', ',', 'but', 'only', 'if', 'no', 'palestinians', 'from', 'east', 'jerusalem', 'or', 'with', 'any', 'connection', 'to', 'the', 'palestine', 'liberation', 'organization', 'will', 'be', 'there', '.', 'the', 'prime', 'minister', 'said', 'that', 'if', 'it', 'turns', 'out', 'that', 'no', 'palestinians', 'are', 'prepared', 'to', 'come', 'to', 'a', 'conference', 'on', 'such', 'terms', ',', 'israel', 'would', 'be', 'willing', 'to', 'negotiate', 'directly', 'with', 'any', 'arab', 'country', '.', 'but', 'he', 'caution', '##ed', 'both', 'syria', 'and', 'jordan', 'that', 'they', 'must', 'not', 'expect', 'to', 'get', 'any', 'land', 'for', 'peace', '-', '-', 'only', 'peace', 'for', 'peace', '.', 'the', 'thrust', 'of', 'mr', '.', 'sham', '##ir', "'", 's

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [125]:
print(len(period0_train), len(input_ids), len(attention_masks))

899 899 899


In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2020, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2020, test_size=0.05)

###### LSTM

In [0]:
vocab_in_size = tokenizer.vocab_size
embedding_dim = 32
unit = 100
no_labels = len(np.unique(train_labels))
batch_size = 32

In [132]:
# Using Sigmoid Activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='sigmoid'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 128, 32)           976704    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 202       
Total params: 1,030,106
Trainable params: 1,030,106
Non-trainable params: 0
_________________________________________________________________


In [133]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Train on 854 samples, validate on 45 samples
Epoch 1/10
854/854 [==============================] - 6s 8ms/step - loss: 0.4396 - acc: 0.9157 - val_loss: 0.0342 - val_acc: 1.0000
Epoch 2/10
854/854 [==============================] - 5s 6ms/step - loss: 0.2400 - acc: 0.9379 - val_loss: 0.0456 - val_acc: 1.0000
Epoch 3/10
854/854 [==============================] - 5s 6ms/step - loss: 0.2338 - acc: 0.9379 - val_loss: 0.0564 - val_acc: 1.0000
Epoch 4/10
854/854 [==============================] - 5s 6ms/step - loss: 0.2337 - acc: 0.9379 - val_loss: 0.0915 - val_acc: 1.0000
Epoch 5/10
854/854 [==============================] - 5s 6ms/step - loss: 0.2308 - acc: 0.9379 - val_loss: 0.0591 - val_acc: 1.0000
Epoch 6/10
854/854 [==============================] - 5s 6ms/step - loss: 0.2184 - acc: 0.9379 - val_loss: 0.0705 - val_acc: 1.0000
Epoch 7/10
854/854 [==============================] - 6s 6ms/step - loss: 0.2067 - acc: 0.9379 - val_loss: 0.0781 - val_acc: 1.0000
Epoch 8/10
854/854 [===========

In [134]:
# Create sentence and label lists
sentences = period0_test.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period0_test.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'the', 'first', 'universal', 'nation', ':', 'leading', 'indicators', 'and', 'ideas', 'about', 'the', 'surge', 'of', 'america', 'in', 'the', '1990s', 'ben', 'j', '.', 'watt', '##enberg', '/', 'the', 'free', 'press', '/', '41', '##8', 'pp', '.', '$', '22', '.', '95', 'there', 'are', 'only', 'two', 'nations', 'on', 'this', 'earth', 'that', 'are', 'simultaneously', 'fantasies', 'and', 'realities', ',', 'those', 'two', 'being', ',', 'of', 'course', ',', 'the', 'tremendous', 'united', 'states', 'of', 'america', 'and', 'tiny', 'israel', '.', 'maybe', 'that', 'is', 'what', 'makes', 'them', ',', 'even', 'when', 'they', "'", 're', 'wr', '##ang', '##ling', ',', 'such', 'a', 'nice', 'couple', '.', 'if', 'america', 'con', '##notes', 'new', 'beginnings', ',', 'fresh', 'starts', 'for', 'one', 'and', 'all', ',', 'the', 'meanings', 'of', 'zion', 'are', 'too', 'well', 'known', 'to', 'mention', '.', 'the', 'motto', 'of', 'both', 'of', 'these', 'very', 'peculiar', 'u

In [135]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)
evaluate_lstm

48/48 [==============================] - 0s 4ms/step


[0.3465331370631854, 0.9166666666666666]

###### BERT Model

In [0]:
train_labels = train_labels.astype(np.int64)
validation_labels = validation_labels.astype(np.int64)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [139]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [145]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.31
  Training epcoh took: 0:00:20

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.20
  Training epcoh took: 0:00:20

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:00:20

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.11
  Training epcoh took: 0:00:21

Running Validation...
  Accuracy: 1.00
  Validation took: 0:00:00

Training complete!


In [147]:
loss_values

[0.31005677773996637,
 0.19703539626465905,
 0.15298841303835312,
 0.11099545119537248]

In [146]:
len(period0_test)

48

In [148]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(period0_test.shape[0]))

# Create sentence and label lists
sentences = period0_test.sentence.values
labels = period0_test.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length=512
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels.astype(np.int64))

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 48



In [149]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 48 test sentences...
    DONE.


In [150]:
print('Positive samples: %d of %d (%.2f%%)' % (period0_test.label.sum(), len(period0_test.label), (period0_test.label.sum() / len(period0_test.label) * 100.0)))

Positive samples: 4 of 48 (8.33%)


In [151]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [152]:
matthews_set

[0.0, 0.0]

In [153]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [154]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './period0/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./period0/


('./period0/vocab.txt',
 './period0/special_tokens_map.json',
 './period0/added_tokens.json')

#### Period 1

In [156]:
period1 = data[data.period==1]
period1

,sentence,label,period
280,twenty-five years ago phyllis chesler 's book...,0,1
284,it 's tough suddenly changing from one alphab...,0,1
285,ii.s. military personnel of all ranks are fee...,0,1
287,on the redemptive pain of loving the natural ...,0,1
289,""" . . . the last few decades have brought gre...",0,1
...,...,...,...
2804,a new round of the violence that has beset i...,0,1
2805,miami - when maria cueto sees little elian g...,0,1
2806,"while snow pelted denver , aspiring russian ...",0,1
2807,"back in the last century , when he worked fo...",0,1


In [160]:
period1.label.sum()/len(period1)

0.06571936056838366

In [0]:
period1_train, period1_test = train_test_split(period1, random_state=2020, test_size=0.05)

In [158]:
print(len(period1_train), len(period1_test))

534 29


In [159]:
# Create sentence and label lists
sentences = period1_train.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period1_train.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', '!', 'mr', '-', 'russ', '##ert', ':', 'but', 'first', ':', 'former', 'pow', ',', 'now', 'presidential', 'candidate', ',', 'senator', 'john', 'mccain', 'of', 'arizona', ',', 'welcome', '.', '!', 'mr', '-', 'mccain', ':', 'thank', 'you', ',', 'tim', '.', '!', 'mr', '-', 'russ', '##ert', ':', 'your', 'campaign', 'is', 'hot', ',', 'dead', 'even', 'with', 'george', 'bush', 'in', 'new', 'hampshire', '.', 'the', 'cover', 'of', 'time', 'magazine', ',', 'the', 'real', 'mccain', '.', 'but', 'still', 'continuing', 'focus', 'on', 'the', 'hot', 'temper', 'issue', '.', '!', 'mr', '-', 'mccain', ':', 'yes', ',', 'indeed', '.', '!', 'mr', '-', 'russ', '##ert', ':', 'here', "'", 's', 'the', 'new', 'york', 'times', ':', 'mccain', 'releases', 'medical', 'files', 'to', 'counter', 'whisper', 'campaign', '.', 'what', "'", 's', 'the', 'whisper', 'campaign', '?', '!', 'mr', '-', 'mccain', ':', 'i', 'do', 'n', "'", 't', 'know', ',', 'and', 'i', 'hope', 'it', 'goes', 'away

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [163]:
print(len(period1_train), len(input_ids), len(attention_masks))

534 534 534


In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2020, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2020, test_size=0.05)

###### LSTM

In [0]:
vocab_in_size = tokenizer.vocab_size
embedding_dim = 32
unit = 100
no_labels = len(np.unique(train_labels))
batch_size = 32

In [166]:
# Using Sigmoid Activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='sigmoid'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 128, 32)           976704    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 202       
Total params: 1,030,106
Trainable params: 1,030,106
Non-trainable params: 0
_________________________________________________________________


In [167]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Train on 507 samples, validate on 27 samples
Epoch 1/10
507/507 [==============================] - 5s 9ms/step - loss: 0.5705 - acc: 0.9152 - val_loss: 0.2681 - val_acc: 0.9259
Epoch 2/10
507/507 [==============================] - 3s 6ms/step - loss: 0.2434 - acc: 0.9369 - val_loss: 0.2641 - val_acc: 0.9259
Epoch 3/10
507/507 [==============================] - 3s 6ms/step - loss: 0.2383 - acc: 0.9369 - val_loss: 0.2649 - val_acc: 0.9259
Epoch 4/10
507/507 [==============================] - 3s 6ms/step - loss: 0.2353 - acc: 0.9369 - val_loss: 0.2648 - val_acc: 0.9259
Epoch 5/10
507/507 [==============================] - 3s 7ms/step - loss: 0.2373 - acc: 0.9369 - val_loss: 0.2658 - val_acc: 0.9259
Epoch 6/10
507/507 [==============================] - 3s 6ms/step - loss: 0.2301 - acc: 0.9369 - val_loss: 0.2660 - val_acc: 0.9259
Epoch 7/10
507/507 [==============================] - 3s 6ms/step - loss: 0.2261 - acc: 0.9369 - val_loss: 0.2692 - val_acc: 0.9259
Epoch 8/10
507/507 [===========

In [168]:
# Create sentence and label lists
sentences = period0_test.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period0_test.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'the', 'first', 'universal', 'nation', ':', 'leading', 'indicators', 'and', 'ideas', 'about', 'the', 'surge', 'of', 'america', 'in', 'the', '1990s', 'ben', 'j', '.', 'watt', '##enberg', '/', 'the', 'free', 'press', '/', '41', '##8', 'pp', '.', '$', '22', '.', '95', 'there', 'are', 'only', 'two', 'nations', 'on', 'this', 'earth', 'that', 'are', 'simultaneously', 'fantasies', 'and', 'realities', ',', 'those', 'two', 'being', ',', 'of', 'course', ',', 'the', 'tremendous', 'united', 'states', 'of', 'america', 'and', 'tiny', 'israel', '.', 'maybe', 'that', 'is', 'what', 'makes', 'them', ',', 'even', 'when', 'they', "'", 're', 'wr', '##ang', '##ling', ',', 'such', 'a', 'nice', 'couple', '.', 'if', 'america', 'con', '##notes', 'new', 'beginnings', ',', 'fresh', 'starts', 'for', 'one', 'and', 'all', ',', 'the', 'meanings', 'of', 'zion', 'are', 'too', 'well', 'known', 'to', 'mention', '.', 'the', 'motto', 'of', 'both', 'of', 'these', 'very', 'peculiar', 'u

In [169]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)
evaluate_lstm

48/48 [==============================] - 0s 3ms/step


[0.29820192356904346, 0.9166666666666666]

###### BERT Model

In [0]:
train_labels = train_labels.astype(np.int64)
validation_labels = validation_labels.astype(np.int64)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [173]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [179]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.33
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.23
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.22
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.21
  Training epcoh took: 0:00:12

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

Training complete!


In [180]:
loss_values

[0.32729432824999094,
 0.23155721370130777,
 0.21828558202832937,
 0.20521679567173123]

In [181]:
len(period1_test)

29

In [182]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(period1_test.shape[0]))

# Create sentence and label lists
sentences = period1_test.sentence.values
labels = period1_test.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length=512
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels.astype(np.int64))

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 29



In [183]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 29 test sentences...
    DONE.


In [184]:
print('Positive samples: %d of %d (%.2f%%)' % (period1_test.label.sum(), len(period1_test.label), (period1_test.label.sum() / len(period1_test.label) * 100.0)))

Positive samples: 3 of 29 (10.34%)


In [185]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [186]:
matthews_set

[0.0]

In [187]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [188]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './period1/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./period1/


('./period1/vocab.txt',
 './period1/special_tokens_map.json',
 './period1/added_tokens.json')

#### Period 2

In [11]:
period2 = data[data.period==2]
period2

,sentence,label,period
405,these kick-ass activists are kicking ass // w...,0,2
406,loung ung 's journey photograph preceding pag...,0,2
407,women find new shelter from the sex industry ...,0,2
412,"tulips . dot-coms . hey , manias happen . but...",1,2
413,as dvds take over more and more of the home v...,0,2
...,...,...,...
3108,we must turn to the past for a film as innoc...,0,2
3109,iraqi expatriates are clamoring to cast ball...,0,2
3110,it begins with news reports of freakish ligh...,0,2
3111,"havre , mont . - surrounded by fields of kne...",0,2


In [12]:
period2.label.sum()/len(period2)

0.08053691275167785

In [0]:
period2_train, period2_test = train_test_split(period2, random_state=2020, test_size=0.05)

In [14]:
print(len(period2_train), len(period2_test))

566 30


In [17]:
# Create sentence and label lists
sentences = period2_train.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period2_train.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'section', ':', '10', 'miles', 'square', 'the', 'last', 'low', '-', 'rent', 'office', 'building', 'in', 'downtown', 'd', '.', 'c', '.', 'smoke', '##s', 'out', 'the', 'national', 'organization', 'for', 'women', ',', 'us', '-', 'ukraine', 'foundation', ',', 'a', 'private', 'eye', ',', 'and', 'the', 'washington', 'monthly', '.', 'now', 'it', 'can', 'be', 'told', '.', 'it', 'is', 'not', '-', '-', 'and', 'never', 'has', 'been', '-', '-', 'tricky', 'to', 'sneak', 'into', 'the', 'washington', 'monthly', "'", 's', 'downtown', 'office', 'building', 'on', 'the', 'come', '##r', 'of', 'h', 'and', '15th', 'streets', ',', 'one', 'block', 'from', 'the', 'sniper', '##s', 'atop', 'the', 'white', 'house', '.', 'on', 'a', 'grey', 'columbus', 'day', ',', 'as', 'on', 'all', 'federal', 'holidays', ',', 'the', 'monthly', 'was', 'open', 'for', 'business', ',', 'but', 'the', 'building', "'", 's', 'front', 'door', 'was', 'bolted', '.', 'when', 'i', 'realized', 'i', "'", 'd

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [19]:
print(len(period2_train), len(input_ids), len(attention_masks))

566 566 566


In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2020, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2020, test_size=0.05)

###### LSTM

In [0]:
vocab_in_size = tokenizer.vocab_size
embedding_dim = 32
unit = 100
no_labels = len(np.unique(train_labels))
batch_size = 32

In [24]:
# Using Sigmoid Activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='sigmoid'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 32)           976704    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 1,030,106
Trainable params: 1,030,106
Non-trainable params: 0
_________________________________________________________________


In [25]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 537 samples, validate on 29 samples
Epoch 1/10





537/537 [==============================] - 4s 8ms/step - loss: 0.5440 - acc: 0.9050 - val_loss: 0.2680 - val_acc: 0.9310
Epoch 2/10
537/537 [==============================] - 3s 6ms/step - loss: 0.2973 - acc: 0.9181 - val_loss: 0.2579 - val_acc: 0.9310
Epoch 3/10
537/537 [==============================] - 3s 6ms/step - loss: 0.2845 - acc: 0.9181 - val_loss: 0.2514 - val_acc: 0.9310
Epoch 4/10
537/537 [==============================] - 3s 6ms/step - loss: 0.2824 - acc: 0.9181 - val_loss: 0.2519 - val_acc: 0.9310
Epoch 5/10
537/537 [==============================] - 3s 6ms/step - loss: 0.2795 - acc: 0.9181 - val_loss: 0.2523 - val_acc: 0.9310
Epoch 6/10
537/537 [==============================] - 3s 6ms/step - loss: 0.2737 - acc: 0.9181 - val_loss: 0.2517 - val_acc: 0.9310
Epoch 7/10
537/537 [==============================] - 4s 7ms/

In [26]:
# Create sentence and label lists
sentences = period2_test.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period2_test.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'nobody', "'", 's', 'paying', 'much', 'attention', 'to', 'the', 'activity', 'in', 'the', 'tucked', '-', 'away', 'corner', 'of', 'car', '##ib', '##ou', 'coffee', 'on', 'peach', '##tree', 'road', '.', 'na', '##ges', '##h', 'ku', '##ku', '##no', '##or', ',', 'a', 'film', 'director', 'who', 'lives', 'in', 'brook', '##haven', ',', 'is', 'nearly', 'pinned', 'to', 'the', 'shop', "'", 's', 'camel', '-', 'colored', 'wall', '.', 'he', "'", 's', 'getting', 'his', 'picture', 'taken', 'by', 'a', 'journal', '-', 'constitution', 'photographer', '.', 'the', 'car', '##ib', '##ou', 'staff', 'keeps', 'on', 'making', 'mo', '##cha', '##s', 'and', 'la', '##ttes', '.', 'the', 'few', 'afternoon', 'patrons', 'hardly', 'look', 'up', 'from', 'their', 'java', 'cups', '.', 'obviously', ',', 'to', 'them', ',', 'it', "'", 's', 'not', 'as', 'if', 'this', 'guy', 'is', 'ba', '##z', 'lu', '##hr', '##mann', '(', '"', 'mo', '##ulin', 'rouge', '"', ')', ',', 'christopher', 'nolan', '(

In [27]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)
evaluate_lstm

30/30 [==============================] - 0s 3ms/step


[0.2344624549150467, 0.9333333373069763]

###### BERT Model

In [0]:
train_labels = train_labels.astype(np.int64)
validation_labels = validation_labels.astype(np.int64)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [31]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [37]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.32
  Training epcoh took: 0:00:13

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.28
  Training epcoh took: 0:00:13

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.27
  Training epcoh took: 0:00:13

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.26
  Training epcoh took: 0:00:13

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:00

Training complete!


In [38]:
loss_values

[0.3214767671683255,
 0.2836875056519228,
 0.27270139227895174,
 0.2581087201833725]

In [39]:
len(period2_test)

30

In [40]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(period2_test.shape[0]))

# Create sentence and label lists
sentences = period2_test.sentence.values
labels = period2_test.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length=512
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels.astype(np.int64))

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 30



In [41]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 30 test sentences...
    DONE.


In [42]:
print('Positive samples: %d of %d (%.2f%%)' % (period2_test.label.sum(), len(period2_test.label), (period2_test.label.sum() / len(period2_test.label) * 100.0)))

Positive samples: 2 of 30 (6.67%)


In [43]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [44]:
matthews_set

[0.0]

In [45]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [46]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './period2/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./period2/


('./period2/vocab.txt',
 './period2/special_tokens_map.json',
 './period2/added_tokens.json')

#### Period 3

In [47]:
period3 = data[data.period==3]
period3

,sentence,label,period
567,"once again , ideology trumps good science and...",0,3
568,women overcame years of gender apartheid-and ...,0,3
569,"again , rape surfaces as an international war...",0,3
575,"although they seem day-and-night different , ...",0,3
578,bishop t.d . jakes wants his flock not only t...,0,3
...,...,...,...
3364,(end-video-clip) (end-video-clip) (end-video-...,0,3
3365,a recent u.s. supreme court ruling that nonci...,0,3
3366,"las vegas -- aces are aces , but pairs of nin...",0,3
3367,brown # bread molasses and salted butterscotc...,0,3


In [48]:
period3.label.sum()/len(period3)

0.09234828496042216

In [0]:
period3_train, period3_test = train_test_split(period3, random_state=2020, test_size=0.05)

In [50]:
print(len(period3_train), len(period3_test))

360 19


In [51]:
# Create sentence and label lists
sentences = period3_train.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period3_train.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'in', 'july', ',', 'lebanon', "'", 's', 'leadership', 'finally', 'formed', 'a', 'so', '-', 'member', 'cabinet', ',', 'with', 'the', 'hezbollah', '-', 'led', 'opposition', 'gaining', 'decisive', 'new', 'powers', '.', 'this', 'is', 'the', 'nation', "'", 's', 'first', 'united', 'government', 'since', 'november', '2006', ',', 'when', 'an', 'opposition', 'walk', '##out', 'triggered', 'an', '18', '##mont', '##h', 'political', 'crisis', ',', 'including', 'six', 'months', 'during', 'which', 'parliament', '-', 'unable', 'to', 'pass', 'legislation', 'for', 'more', 'than', 'a', 'year', '-', 'could', 'n', "'", 't', 'agree', 'on', 'a', 'president', '.', 'women', 'have', 'been', 'hit', 'hard', 'by', 'this', 'crisis', ',', 'the', 'expansion', 'of', 'their', 'rights', 'put', 'on', 'hold', '.', 'but', 'the', 'women', "'", 's', 'movement', ',', 'eager', 'to', 'make', 'gender', 'equality', 'a', 'priority', 'for', 'this', 'new', 'cabinet', ',', 'has', 'already', 'mob

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [53]:
print(len(period3_train), len(input_ids), len(attention_masks))

360 360 360


In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2020, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2020, test_size=0.05)

###### LSTM

In [0]:
vocab_in_size = tokenizer.vocab_size
embedding_dim = 32
unit = 100
no_labels = len(np.unique(train_labels))
batch_size = 32

In [56]:
# Using Sigmoid Activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='sigmoid'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 128, 32)           976704    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 1,030,106
Trainable params: 1,030,106
Non-trainable params: 0
_________________________________________________________________


In [57]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Train on 342 samples, validate on 18 samples
Epoch 1/10
342/342 [==============================] - 3s 9ms/step - loss: 0.6645 - acc: 0.8801 - val_loss: 0.5758 - val_acc: 0.8889
Epoch 2/10
342/342 [==============================] - 2s 7ms/step - loss: 0.3832 - acc: 0.9123 - val_loss: 0.3572 - val_acc: 0.8889
Epoch 3/10
342/342 [==============================] - 2s 7ms/step - loss: 0.2952 - acc: 0.9123 - val_loss: 0.3493 - val_acc: 0.8889
Epoch 4/10
342/342 [==============================] - 2s 6ms/step - loss: 0.2971 - acc: 0.9123 - val_loss: 0.3518 - val_acc: 0.8889
Epoch 5/10
342/342 [==============================] - 2s 6ms/step - loss: 0.2966 - acc: 0.9123 - val_loss: 0.3534 - val_acc: 0.8889
Epoch 6/10
342/342 [==============================] - 2s 6ms/step - loss: 0.2917 - acc: 0.9123 - val_loss: 0.3472 - val_acc: 0.8889
Epoch 7/10
342/342 [==============================] - 2s 6ms/step - loss: 0.2824 - acc: 0.9123 - val_loss: 0.3552 - val_acc: 0.8889
Epoch 8/10
342/342 [===========

In [58]:
# Create sentence and label lists
sentences = period3_test.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period3_test.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'in', 'white', 'house', 'meetings', 'beginning', 'monday', ',', 'president', 'ho', '##s', '##ni', 'mu', '##bara', '##k', 'of', 'egypt', 'is', 'expected', 'to', 'tell', 'the', 'obama', 'administration', 'that', 'arab', 'nations', 'want', 'peace', ',', 'but', 'are', 'unwilling', 'to', 'ab', '##ide', 'mr', '.', 'obama', "'", 's', 'call', 'to', 'make', 'good', '-', 'faith', 'concessions', 'to', 'israel', 'until', 'israel', 'takes', 'tangible', 'steps', 'like', 'freezing', 'settlements', ',', 'an', 'egyptian', 'official', 'said', '.', '#', 'as', 'part', 'of', 'its', 'effort', 'to', 'res', '##us', '##cit', '##ate', 'the', 'peace', 'process', ',', 'the', 'obama', 'administration', 'has', 'asked', 'arab', 'countries', 'to', 'make', 'small', 'but', 'symbolic', 'gestures', 'to', 'normal', '##ize', 'relations', 'with', 'israel', ',', 'like', 'allowing', 'planes', 'to', 'fly', 'through', 'their', 'airspace', 'or', 'improving', 'cultural', 'ties', '.', 'the', 

In [59]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)
evaluate_lstm

19/19 [==============================] - 0s 4ms/step


[0.4725702106952667, 0.8421052694320679]

###### BERT Model

In [0]:
train_labels = train_labels.astype(np.int64)
validation_labels = validation_labels.astype(np.int64)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [63]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [69]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.38
  Training epcoh took: 0:00:08

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.31
  Training epcoh took: 0:00:08

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.30
  Training epcoh took: 0:00:08

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.29
  Training epcoh took: 0:00:08

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:00

Training complete!


In [70]:
loss_values

[0.37997413900765503,
 0.30528532578186557,
 0.29806255075064575,
 0.2937267639420249]

In [71]:
len(period3_test)

19

In [73]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(period3_test.shape[0]))

# Create sentence and label lists
sentences = period3_test.sentence.values
labels = period3_test.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length=512
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels.astype(np.int64))

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 19



In [74]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 19 test sentences...
    DONE.


In [75]:
print('Positive samples: %d of %d (%.2f%%)' % (period3_test.label.sum(), len(period3_test.label), (period3_test.label.sum() / len(period3_test.label) * 100.0)))

Positive samples: 3 of 19 (15.79%)


In [77]:
true_labels

[array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])]

In [78]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [79]:
matthews_set

[0.0]

In [80]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [81]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './period3/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./period3/


('./period3/vocab.txt',
 './period3/special_tokens_map.json',
 './period3/added_tokens.json')

#### Period 4

In [11]:
period4 = data[data.period==4]
period4

,sentence,label,period
738,calls for the creation of a no-fly zone over ...,0,4
739,"he 's not here in "" welcome to the cafeteria ...",0,4
740,"as a graduate student in theology , i lived i...",0,4
741,"get ready for more undernourished infants , d...",1,4
742,"sunday , march 27 exodus 17:1-7 translated li...",1,4
...,...,...,...
3987,"!kelly-mcevers# oh , thanks , nice to meet yo...",0,4
3988,!margaret-warner# and i 'm margaret warner . ...,0,4
3989,"!judy-woodruff# the syrian opposition , u.n ....",0,4
3990,!charlie-rose : good morning . it is friday d...,0,4


In [12]:
period4.label.sum()/len(period4)

0.1828793774319066

In [0]:
period4_train, period4_test = train_test_split(period4, random_state=2020, test_size=0.05)

In [14]:
print(len(period4_train), len(period4_test))

488 26


In [15]:
# Create sentence and label lists
sentences = period4_train.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period4_train.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', 'essay', 'the', 'politics', '#', 'the', 'labour', 'leader', "'", 's', 'mission', 'to', 'res', '##ha', '##pe', 'capitalism', 'in', 'britain', '#', 'it', 'was', 'the', 'kind', 'of', 'challenge', 'that', 'made', 'tony', 'blair', 'and', 'gordon', 'brown', 'wince', '.', '"', 'when', 'are', 'you', 'going', 'to', 'bring', 'back', 'socialism', '?', '"', 'called', 'the', 'man', 'in', 'the', 'crowd', 'to', 'ed', 'mil', '##iba', '##nd', 'at', 'a', 'rally', 'in', 'brighton', 'on', 'the', 'eve', 'of', 'his', 'party', "'", 's', 'annual', 'conference', 'in', 'september', '.', 'the', 'labour', 'leader', 'paused', 'before', 'offering', 'a', 'response', 'that', 'his', 'two', 'predecessors', 'would', 'not', 'have', 'dared', 'deliver', ':', '"', 'that', "'", 's', 'what', 'we', "'", 're', 'doing', ',', 'sir', '.', '"', '#', 'when', ',', 'later', 'that', 'week', ',', 'mil', '##iba', '##nd', 'used', 'his', 'conference', 'speech', 'to', 'call', 'for', 'a', 'freeze', 'in'

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [17]:
print(len(period4_train), len(input_ids), len(attention_masks))

488 488 488


In [0]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2020, test_size=0.05)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2020, test_size=0.05)

###### LSTM

In [0]:
vocab_in_size = tokenizer.vocab_size
embedding_dim = 32
unit = 100
no_labels = len(np.unique(train_labels))
batch_size = 32

In [20]:
# Using Sigmoid Activation
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_in_size, embedding_dim, input_length=MAX_LEN))
model_lstm.add(LSTM(unit))
model_lstm.add(Dense(no_labels, activation='sigmoid'))
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 32)           976704    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 1,030,106
Trainable params: 1,030,106
Non-trainable params: 0
_________________________________________________________________


In [21]:
history_lstm = model_lstm.fit(train_inputs, train_labels, 
                              epochs=10,
                              validation_data=(validation_inputs, validation_labels),
                              batch_size=batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 463 samples, validate on 25 samples
Epoch 1/10





463/463 [==============================] - 4s 9ms/step - loss: 0.6388 - acc: 0.7927 - val_loss: 0.5171 - val_acc: 0.8400
Epoch 2/10
463/463 [==============================] - 3s 7ms/step - loss: 0.4991 - acc: 0.8164 - val_loss: 0.4698 - val_acc: 0.8400
Epoch 3/10
463/463 [==============================] - 3s 6ms/step - loss: 0.4686 - acc: 0.8164 - val_loss: 0.4395 - val_acc: 0.8400
Epoch 4/10
463/463 [==============================] - 3s 6ms/step - loss: 0.4556 - acc: 0.8164 - val_loss: 0.4443 - val_acc: 0.8400
Epoch 5/10
463/463 [==============================] - 3s 7ms/step - loss: 0.3969 - acc: 0.8164 - val_loss: 0.4423 - val_acc: 0.8400
Epoch 6/10
463/463 [==============================] - 3s 6ms/step - loss: 0.2342 - acc: 0.8747 - val_loss: 0.6707 - val_acc: 0.8400
Epoch 7/10
463/463 [==============================] - 3s 6ms/

In [22]:
# Create sentence and label lists
sentences = period4_test.sentence.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = period4_test.label.values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

Tokenize the first sentence:
['[CLS]', '!', 'gwen', '-', 'if', '##ill', '#', 'president', 'obama', 'addresses', 'the', 'nation', 'tonight', 'in', 'his', 'sixth', 'state', 'of', 'the', 'union', ',', 'and', 'for', 'the', 'first', 'time', 'before', 'a', 'house', 'and', 'senate', 'under', 'republican', 'control', '.', 'good', 'evening', '.', 'i', "'", 'm', 'gwen', 'if', '##ill', '.', '!', 'judy', '-', 'wood', '##ruff', '#', 'and', 'i', "'", 'm', 'judy', 'wood', '##ruff', '.', 'we', 'bring', 'you', 'special', 'coverage', 'from', 'the', 'u', '.', 's', '.', 'capitol', 'and', 'analysis', 'from', 'mark', 'shields', 'and', 'david', 'brooks', '.', '!', 'gwen', '-', 'if', '##ill', '#', 'plus', ':', 'islamic', 'state', 'militants', 'threaten', 'to', 'kill', 'two', 'japanese', 'hostages', 'unless', 'paid', 'a', '$', '200', 'million', 'ransom', '.', '!', 'judy', '-', 'wood', '##ruff', '#', 'and', 'a', 'california', 'public', 'school', 'shifts', 'its', 'focus', 'to', 'life', 'lessons', ',', 'preparing

In [23]:
evaluate_lstm = model_lstm.evaluate(input_ids, labels, batch_size=batch_size)
evaluate_lstm

26/26 [==============================] - 0s 3ms/step


[0.5085266828536987, 0.9230769276618958]

###### BERT Model

In [0]:
train_labels = train_labels.astype(np.int64)
validation_labels = validation_labels.astype(np.int64)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [27]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [0]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [33]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.52
  Training epcoh took: 0:00:11

Running Validation...
  Accuracy: 0.84
  Validation took: 0:00:00

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.47
  Training epcoh took: 0:00:11

Running Validation...
  Accuracy: 0.84
  Validation took: 0:00:00

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.46
  Training epcoh took: 0:00:11

Running Validation...
  Accuracy: 0.84
  Validation took: 0:00:00

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.43
  Training epcoh took: 0:00:11

Running Validation...
  Accuracy: 0.84
  Validation took: 0:00:00

Training complete!


In [34]:
loss_values

[0.5166027386983235,
 0.4744277199109395,
 0.46440380613009136,
 0.4312203774849574]

In [35]:
len(period4_test)

26

In [36]:
import pandas as pd

# Load the dataset into a pandas dataframe.
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(period4_test.shape[0]))

# Create sentence and label lists
sentences = period4_test.sentence.values
labels = period4_test.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length=512
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels.astype(np.int64))

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 26



In [37]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 26 test sentences...
    DONE.


In [38]:
print('Positive samples: %d of %d (%.2f%%)' % (period4_test.label.sum(), len(period4_test.label), (period4_test.label.sum() / len(period4_test.label) * 100.0)))

Positive samples: 5 of 26 (19.23%)


In [39]:
true_labels

[array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 1, 0])]

In [40]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [41]:
matthews_set

[0.0]

In [42]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [44]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './period4/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./period4/


('./period4/vocab.txt',
 './period4/special_tokens_map.json',
 './period4/added_tokens.json')

### Text Generation

In [0]:
torch.cuda.empty_cache() 

In [0]:
!python /content/run_language_modelling.py --output_dir=output_gpt_trump --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=/content/train_text_trump --do_eval --eval_data_file=/content/test_text_trump --per_gpu_train_batch_size=1 --per_gpu_eval_batch_size=1

03/08/2020 07:38:57 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
03/08/2020 07:38:58 - INFO - filelock -   Lock 140686358784712 acquired on /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942.lock
03/08/2020 07:38:58 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp3_kyg7wp
Downloading: 100% 224/224 [00:00<00:00, 201kB/s]
03/08/2020 07:38:58 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json in cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942
03/08/2020 07:38:58 - INFO

In [0]:
!python run_generation.py --model_type=gpt2 --model_name_or_path=/content/output_gpt_trump

03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   Model name '/content/output_gpt_trump' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming '/content/output_gpt_trump' is a path, a model identifier, or url to a directory containing tokenizer files.
03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   Didn't find file /content/output_gpt_trump/added_tokens.json. We won't load it.
03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   loading file /content/output_gpt_trump/vocab.json
03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   loading file /content/output_gpt_trump/merges.txt
03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   loading file None
03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   loading file /content/output_gpt_trump/special_tokens_map.json
03/08/2020 07:43:17 - INFO - transformers.tokenization_utils -   loading file /content/output_gpt_tr

### Save Files


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'pytorch_model.bin'})
uploaded.SetContentFile('output_roberta_US/pytorch_model.bin')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1w4rzAyObL6YRo0TOmic7Cwbx1cfSPUPB
